AHMED ALGHAMDI
FIRAS ALZHRANI

# CREATE TABLES USING PYTHON AND SQL
Creating The Tables Books, Members, and Borrowing using SQLite in Python.

In [2]:
import sqlite3

conn = sqlite3.connect('library.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Books (
        id INTEGER PRIMARY KEY,
        title TEXT,
        author TEXT,
        isbn TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Members (
        id INTEGER PRIMARY KEY,
        name TEXT,
        email TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Borrowing (
        id INTEGER PRIMARY KEY,
        member_id INTEGER,
        book_id INTEGER,
        borrowed_date DATE,
        return_date DATE,
        FOREIGN KEY (member_id) REFERENCES Members(id),
        FOREIGN KEY (book_id) REFERENCES Books(id)
    )
''')

conn.commit()
conn.close()

print("Tables created successfully.")

Tables created successfully.


# ADD RECORDS TO THE TABLE
Adding A New Book, A New Member, And A New Borrowing To The Tables.

In [3]:
conn = sqlite3.connect('library.db')
cursor = conn.cursor()

cursor.execute('''
    INSERT INTO Books (title, author, isbn) VALUES (?, ?, ?)
''', ('BOOK', 'J.D. Salinger', '9780316769488'))
book_id = cursor.lastrowid

cursor.execute('''
    INSERT INTO Members (name, email) VALUES (?, ?)
''', ('Ahmed', 'Ahmed@gmail.com'))
member_id = cursor.lastrowid

cursor.execute('''
    INSERT INTO Borrowing (member_id, book_id, borrowed_date, return_date) VALUES (?, ?, ?, ?)
''', (member_id, book_id, '2024-07-02', '2024-08-02'))
borrowing_id = cursor.lastrowid

conn.commit()
conn.close()

print("Records added successfully.")

Records added successfully.


# PREVINTING SQL INJECTIONS
To Prevent SQL injection, I Should Always Use Parameterized Queries. Heres How You Can Fix The Vulnerable Code:

In [4]:
safe_variable = (1,)
query = "SELECT * FROM Members WHERE id = ?"

conn = sqlite3.connect('library.db')
cursor = conn.cursor()
cursor.execute(query, safe_variable)
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

(1, 'John Doe', 'johndoe@example.com')
